In [7]:
import pygame
import time
import random
from scipy.spatial import distance 
import cProfile

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [41]:
def comidas(com):
    comida = {
        'carne' : 50,
        'mora' : 8,
        'manzana' : 20,
    }
    return comida[com]

In [2]:
class Entorno: # Clase para objetos que forman parte de algún bioma o son interactuables
    def __init__(self, tipo, posicion, energia,recuperacion):
        self.tipo = tipo
        self.posicion = posicion  # (x, y) tuple
        self.energia = energia
        self.recuperacion = int(recuperacion/4)
        self.recuperacion_maxima = recuperacion
        
    def disponible(self):
        return self.recuperacion == 0

    def cooldown(self):
        if self.recuperacion > 0:
            self.recuperacion -= 1
    def __str__(self):
        return f"_________ Tipo: {self.tipo}, Posición: {self.posicion}, CD: {self.recuperacion}, Energía: {self.energia}"

In [3]:
class Entorno_complejo: # Elementos de trabajo más complejos como Minas, Molinos, comercios etc.
    def __init__(self, tipo, posicion,cantidad,recuperacion):
        self.tipo = tipo # String
        self.posicion = posicion  # (x, y) tuple
        self.cantidad = cantidad
        self.recuperacion = int(recuperacion/4)
        self.recuperacion_maxima = recuperacion # El tiempo que tarda en volver a usarse este entorno
    def disponible(self):
        return self.recuperacion == 0

    def cooldown(self):
        if self.recuperacion > 0:
            self.recuperacion -= 1
            
    def destruir(self):
        if self.cantidad <= 0:
            mundo.entornos.remove(self)
            
    def __str__(self):
        return f"___ Tipo: {self.tipo}, Posición: {self.posicion}, CD: {self.recuperacion}, Energía: {self.energia}"

In [42]:
class Humano:
    def __init__(self, posicion, energia, inventario = None, profesion = None):
        self.profesion = profesion # Determina el patrón de acciones
        self.posicion = posicion  # (x, y)
        self.energia = energia
        self.energia_maxima = 300  # Energía máxima que puede tener el organismo
        self.inventario = inventario if inventario is not None else {} # {objeto : cantidad}
        self.hogar = None # [(x, y), tamaño] de su hogar
        self.lugares_conocidos = {} # Coordenadas de los biomas que sabe que existen
        self.sin_comer = False # Determina si está sin comer xdxdxd
        #self.genero = genero # M o F para mejorar lo de reproducción a futuro
    
    def acciones(self,mundo):
        if self.hogar == None: # Es para declarar el hogar de cada humano, pero se cambiará
            for hogar in mundo.hogares:
                if distance.euclidean(hogar.posicion,self.posicion)-hogar.tamaño <=0:
                    self.hogar = [hogar.posicion,hogar.tamaño]
        if descansar():
            self.sin_comer = True
            return
        mover()
        cazar(mundo)
        descargar_inventario()
        agregar_inventario()
        alimentar()
        morir()
        nuevo_humano = reproducir()
        if nuevo_humano:
            nuevos_humanos.append(nuevo_humano)
        self.humanos.extend(nuevos_humanos)
            
    def mover(self):
        # raise NotImplementedError("Esto se debe cambiar para manejar herencia")
        if self.profesion == "cazador": 
            if mundo.ciclos <= 45: # Horario laboral
                if 'bosque' in self.lugares_conocidos:
                    objetivo = self.lugares_conocidos['bosque'][random.randint(0,len(self.lugares_conocidos['bosque'])-1)]
                    res = [objetivo[i]-self.posicion[i] for i in range(2)]
                    direccion = [-5 if i < 0 else 5 if i > 0 else 0 for i in res]
                    dx,dy = direccion[0],direccion[1]
                else:
                    dx,dy = random.choice([-5, 0, 5]),random.choice([-5, 0, 5])
            else:
                if self.hogar: # Vuelve al hogar después del trabajo
                    objetivo = self.hogar
                    res = [objetivo[i]-self.posicion[i] for i in range(2)]
                    direccion = [-5 if i < 0 else 5 if i > 0 else 0 for i in res]
                    dx,dy = direccion[0],direccion[1]
                else: 
                    dx,dy = random.choice([-5, 0, 5]),random.choice([-5, 0, 5])
        else:
            dx,dy = random.choice([-5, 0, 5]),random.choice([-5, 0, 5])
        self.posicion = (self.posicion[0] + dx, self.posicion[1] + dy)
        # Si no se mueve pierde menos energia
        if dx == 0 and dy == 0:
            self.energia = round(self.energia - .2,1)
        else:
            self.energia = round(self.energia - 1,1)

    def alimentar(self):
        if mundo.ciclos >= 70 and self.sin_comer: # Horario para comer y aún sin comer
            if distance.euclidean(self.hogar[0],self.posicion)-self.hogar[1] <=0: # Está en el hogar
                for hogar in mundo.hogares:
                    if hogar.posicion == self.hogar[0]:
                        if hogar.inventario: # Revisa que haya objetos en el inventario del hogar
                            objeto = random.choice(list(hogar.inventario.keys())) # Elige uno
                            n_comer = random.randint(1,2) # Decide cuántos va a comer (falta comprobar que haya sufucientes xd)
                            hogar.inventario[objeto] -= n_comer # Resta los que se comió
                            if hogar.inventario[objeto] <= 0: # Si se acaba ese objeto o elimina del i
                                del hogar.inventario[objeto]
                            self.energia = round(self.energía + comidas(objeto)*n_comer,1)
                            self.sin_comer == False
                
    def reproducir(self):
        # Se reproduce si la energía es mayor que un umbral
        if self.energia > 250:
            self.energia //= 2  # Reduce la energía a la mitad
            hijo_posicion = (self.posicion[0] + random.choice([-5, 5]),
                             self.posicion[1] + random.choice([-5, 5]))
            return Humano(hijo_posicion, self.energia)
        return None

    def cazar(self, mundo):
        # Los cazadores cazan zorros o conejos en el rango, solamente uno por ciclo
        if self.profesion == "cazador": 
            for organismo in mundo.organismos:
                if organismo.tipo == "zorro" and self.radio_busqueda(organismo,20):
                    self.agregar_inventario("carne",random.randint(4, 7))
                    mundo.organismos.remove(organismo)
                    break
                if organismo.tipo == "conejo" and self.radio_busqueda(organismo,20):
                    self.agregar_inventario("carne",random.randint(1, 3))
                    mundo.organismos.remove(organismo)
                    break
    def morir(self):
        if self.energia <= 0: # Se murioooooo
            mundo.humanos.remove(self)

    def agregar_inventario(self,objeto,cantidad):
        if objeto in self.inventario.keys(): # Verifica si ya tiene una cantidad de ese objeto
            self.inventario[objeto] += cantidad # si es así, suma la cantidad
        else:
            self.inventario[objeto] = cantidad # si no, agrega el objeto y la cantidad

    def descargar_inventario(self):
        for hogar in mundo.hogares:
            if distance.euclidean(hogar.posicion,self.posicion)-hogar.tamaño <=0: # Verifica si está en un hogar 
                for objeto in self.inventario:
                    if objeto in hogar.inventario.keys():# Verifica si el hogar ya tiene una cantidad de ese objeto
                        hogar.inventario[objeto] += self.inventario[objeto] # si es así, suma la cantidad
                    else:
                        hogar.inventario[objeto] = self.inventario[objeto] # si no, agrega el objeto y la cantidad
                self.inventario = {} # Vacía el inventario
                break

    def descansar(self):
        for hogar in mundo.hogares:
            if distance.euclidean(hogar.posicion,self.posicion)-hogar.tamaño <=0:
                if ciclo >= 75:
                    self.energia = round(self.energia + .5,1)
                    dormido = True
                else:
                    dormido = False
                break
        return dormido
        
    def radio_busqueda(self, otro_organismo,radio): # Implementación chafa, luego la cambio
        dx = abs(self.posicion[0] - otro_organismo.posicion[0])
        dy = abs(self.posicion[1] - otro_organismo.posicion[1])
        return dx <= radio and dy <= radio

    def __str__(self):
        return f"Profesion: {self.profesion}, Posición: {self.posicion}, Energia: {self.energia}, Inventario: {self.inventario}"


In [5]:
class Organismo:
    def __init__(self, tipo, posicion, energia):
        self.tipo = tipo
        self.posicion = posicion  # (x, y)
        self.energia = energia
        self.energia_maxima = 150  # Energía máxima que puede tener el organismo
        self.hogar = None
        #self.genero = genero # M o F para mejorar lo de reproducción a futuro

    def mover(self):
        # Reglas para el conejo
        if self.tipo == "conejo":
            best,objetivo = 9999,0 # Inicializamos variables
            for entorno in mundo.entornos:
                if entorno.tipo == "arbusto" and self.radio_busqueda(entorno,25):# por cada arbusto en su radio 
                    distancia = distance.euclidean(self.posicion, entorno.posicion) # calcula distancia
                    cooldown = entorno.recuperacion # cuánto falta para ser comestible
                    energia = entorno.energia # la anergía que da
                    puntos = distancia+(cooldown*3.5)-(energia*.6) # evalúa con esta fórmula
                    if puntos < best: # si este arbusto tiene menos puntos (es mejor)
                        best = puntos # guarda los puntos
                        objetivo = entorno.posicion # guarda la posición
            if objetivo == 0: # si no hay arbusto, se mueve random
                dx = random.choice([-5, 0, 5])
                dy = random.choice([-5, 0, 5])
            else: # si hay arbusto se mueve hacia el arbusto
                res = [objetivo[i]-self.posicion[i] for i in range(2)]
                direccion = [-5 if i < 0 else 5 if i > 0 else 0 for i in res]
                dx,dy = direccion[0],direccion[1]
        # Reglas para el zorro
        if self.tipo == "zorro":
            best,objetivo = 9999,0
            for organismo in mundo.organismos: # Lea el del conejo, es misma lógica
                if organismo.tipo == "conejo" and self.radio_busqueda(organismo, 35):
                    distancia = distance.euclidean(self.posicion,organismo.posicion)
                    energia = organismo.energia
                    puntos = distancia+energia*.4
                    if puntos < best:
                        best = puntos
                        objetivo = organismo.posicion
            if objetivo == 0:
                dx = random.choice([-10, -5, 0, 5, 10])
                dy = random.choice([-10, -5, 0, 5, 10])
            else:
                res = [objetivo[i]-self.posicion[i] for i in range(2)] #aqui falta poner el -10 y el 10 
                direccion = [-5 if i < 0 else 5 if i > 0 else 0 for i in res]
                dx,dy = direccion[0],direccion[1]
        # Si no se mueve pierde menos energia
        if dx == 0 and dy == 0:
            self.energia = round(self.energia - .2,1)
        else:
            self.energia = round(self.energia - 1,1)
        self.posicion = (self.posicion[0] + dx, self.posicion[1] + dy)
        
    def alimentar(self, mundo):
        # Los conejos Se alimentan de arbustos adyacentes
        if self.tipo == "conejo":
            for entorno in mundo.entornos:
                if entorno.tipo == "arbusto" and self.radio_busqueda(entorno,5) and entorno.disponible():
                    self.energia = min(self.energia + entorno.energia + random.randint(-5, 5), self.energia_maxima)
                    entorno.recuperacion = entorno.recuperacion_maxima
                    break

    def reproducir(self):
        # Se reproduce si la energía es mayor que un umbral
        if self.energia > 80:
            self.energia //= 2  # Reduce la energía a la mitad
            hijo_posicion = (self.posicion[0] + random.choice([-5, 5]),
                             self.posicion[1] + random.choice([-5, 5]))
            return Organismo(self.tipo, hijo_posicion, self.energia)
        return None

    def cazar(self, mundo):
        # Los zorros cazan conejos adyacentes
        if self.tipo == "zorro":
            for organismo in mundo.organismos:
                if organismo.tipo == "conejo" and self.radio_busqueda(organismo,10):
                    self.energia = min(self.energia + organismo.energia + random.randint(0, 25), self.energia_maxima)
                    mundo.organismos.remove(organismo)
                    break
    def morir(self):
        if self.energia <= 0:
            mundo.organismos.remove(self)

    def radio_busqueda(self, otro_organismo,radio):
        dx = abs(self.posicion[0] - otro_organismo.posicion[0])
        dy = abs(self.posicion[1] - otro_organismo.posicion[1])
        return dx <= radio and dy <= radio

    def __str__(self):
        return f"*** Tipo: {self.tipo}, Posición: {self.posicion}, Energia: {self.energia}"


In [6]:
class Hogar:
    def __init__(self,tipo,posicion,tamaño,inventario = None):
        self.tipo = tipo
        self.posicion = posicion
        self.tamaño = tamaño
        self.inventario = inventario if inventario is not None else {}
        
    def __str__(self):
        return f"Tipo: {self.tipo}, Inventario: {self.inventario}"

In [7]:
class Mundo:
    def __init__(self, tamaño):
        self.tamaño = tamaño  # (ancho, alto)
        self.humanos = []
        self.organismos = []
        self.entornos = []
        self.hogares = []
        self.ciclos = 0
        self.dias = 0
        self.biomas = {"bosque": []}

    def agregar_organismo(self, organismo):
        self.organismos.append(organismo)

    def agregar_entorno(self, entorno):
        self.entornos.append(entorno)

    def agregar_hogares(self,hogar):
        self.hogares.append(hogar)
        
    def crear_bioma(self, bioma, posicion_bio, cantidad = None, dispersion = None):
        if bioma == "bosque":
            self.biomas["bosque"].append(posicion_bio)
            for i in range(cantidad):
                n = random.randint(0,2)
                self.entornos.append(Entorno(tipo="arbusto",
                                             posicion=(posicion_bio[0]+random.randint(-dispersion, dispersion)*5,
                                                       posicion_bio[1]+random.randint(-dispersion, dispersion)*5),
                                             energia=[20,70,120][n],
                                             recuperacion=[20,55,80][n]))
    
    def actualizar(self):
        self.ciclos+=1
        if self.ciclos == 100:
            self.dias+=1
            self.ciclos = 0
        nuevos_organismos,nuevos_humanos = [],[]
        for entorno in self.entornos:
            entorno.cooldown()
        for humano in self.humanos:
            humano.mover()
            humano.cazar(self)
            humano.descargar_inventario()
            humano.morir()
            nuevo_humano = humano.reproducir()
            if nuevo_humano:
                nuevos_humanos.append(nuevo_humano)
        self.humanos.extend(nuevos_humanos)

        for organismo in self.organismos:
            organismo.mover()
            organismo.alimentar(self)
            organismo.cazar(self)
            organismo.morir()
            nuevo_organismo = organismo.reproducir()
            if nuevo_organismo:
                nuevos_organismos.append(nuevo_organismo)
        self.organismos.extend(nuevos_organismos)

    def mostrar_estado(self):
        print(f"--- Ciclo {self.ciclos} - Día {self.dias} ---")
        for humano in self.humanos:
            print(humano)
        for organismo in self.organismos:
            print(organismo)
        # for entorno in self.entornos:
        #     print(entorno)
    def mostrar_entornos(self):
        print(f"--- Ciclo {self.ciclos} - Día {self.dias} ---")
        for entorno in self.entornos:
            print(entorno)
    def mostrar_hogares(self):
        print(f"--- Ciclo {self.ciclos} - Día {self.dias} ---")
        for hogar in self.hogares:
            print(hogar)
    def mostrar_biomas(self):
        print(f"--- Ciclo {self.ciclos} - Día {self.dias} ---")
        for bioma in self.biomas:
            print(f'{bioma}: {self.biomas[bioma]}')

In [8]:
# Inicializar el mundo y agregar organismos
mundo = Mundo(tamaño=(20, 20))

mundo.agregar_organismo(Organismo(tipo="conejo", posicion=(140, 140), energia=45))
mundo.agregar_organismo(Organismo(tipo="conejo", posicion=(120, 170), energia=45))
mundo.agregar_organismo(Organismo(tipo="zorro", posicion=(120, 130), energia=70))
mundo.humanos.append(Humano(posicion=(120, 150), energia=150, profesion = "cazador"))

mundo.hogares.append(Hogar("campamento",(120,145),10))

mundo.crear_bioma("bosque",(140,150),25,8)

In [17]:
mundo.mostrar_estado()
mundo.actualizar()
mundo.mostrar_estado()

--- Ciclo 7 - Día 0 ---
Profesion: cazador, Posición: (130, 145), Energia: 143.8, Inventario: {}
--- Ciclo 8 - Día 0 ---
Profesion: cazador, Posición: (135, 150), Energia: 142.8, Inventario: {}


In [18]:
mundo.mostrar_biomas()
mundo.mostrar_hogares()

--- Ciclo 8 - Día 0 ---
bosque: [(140, 150)]
--- Ciclo 8 - Día 0 ---
Tipo: campamento, Inventario: {'carne': 10}


In [11]:
mundo.mostrar_entornos()

--- Ciclo 1 - Día 0 ---
_________ Tipo: arbusto, Posición: (175, 135), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (180, 155), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (110, 190), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (125, 160), CD: 4, Energía: 20
_________ Tipo: arbusto, Posición: (115, 145), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (105, 135), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (160, 125), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (125, 185), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (130, 150), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (175, 115), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (125, 145), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (140, 135), CD: 4, Energía: 20
_________ Tipo: arbusto, Posición: (140, 135), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (125, 190), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición

In [66]:
pygame.init()
pantalla = pygame.display.set_mode((200, 200))
clock = pygame.time.Clock()
running = True

while running:
    clock.tick(60)
    for evento in pygame.event.get():
        if evento.type == pygame.QUIT:
            running = False
            pygame.quit()
    if not running:
        break

    pantalla.fill((255, 255, 255))
    for entorno in mundo.entornos:
        pygame.draw.circle(pantalla, (128, 255, 128), entorno.posicion, 2.5)
    for organismo in mundo.organismos:
        if organismo.tipo == "conejo":
            pygame.draw.circle(pantalla, (0, 0, 255), organismo.posicion, 2.5)
        else:
            pygame.draw.circle(pantalla, (255, 0, 0), organismo.posicion, 2.5)
    pygame.display.flip()
    mundo.mostrar_estado()
    mundo.actualizar()
    time.sleep(1)
print("Salida")

--- Ciclo 1 - Día 0 ---
Tipo: conejo, Posición: (105, 100), Energía: 44
Tipo: conejo, Posición: (75, 175), Energía: 44
Tipo: zorro, Posición: (115, 125), Energía: 69
_________ Tipo: arbusto, Posición: (100, 165), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (165, 160), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (135, 150), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (130, 170), CD: 12, Energía: 70
_________ Tipo: arbusto, Posición: (155, 150), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (140, 125), CD: 4, Energía: 20
_________ Tipo: arbusto, Posición: (95, 190), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (100, 130), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (145, 115), CD: 19, Energía: 120
_________ Tipo: arbusto, Posición: (105, 145), CD: 12, Energía: 70
--- Ciclo 2 - Día 0 ---
Tipo: conejo, Posición: (110, 100), Energía: 43
Tipo: conejo, Posición: (70, 170), Energía: 43
Tipo: zorro, Posición: (115, 120), Energía: